<a href="https://colab.research.google.com/github/command404/api-for-sending-alerts-from-gmail/blob/main/gmail_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
!pip install requests



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving credentials.json to credentials.json


In [ ]:
#cell3
import os
import pickle
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'credentials.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please visit this URL to authorize:")
print(auth_url)

# Paste authorization code
code = input("Enter authorization code: ")
flow.fetch_token(code=code)
creds = flow.credentials

# Finally build Gmail service object
service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=273308984561-qg41n2vnoeoccbchsgds8ucsoasfl0bp.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=yeWhua4QeIytgMe2pvoPAgzIHytBug&prompt=consent&access_type=offline
Enter authorization code: 4/1AVGzR1BoTemRWt6QQvtDwLbXPbZAU66ux6T09xQbv-mER1Szl3svXJibLPA


In [ ]:
#Cell  4   import os
import pickle
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'credentials.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)
# This part will require user interaction to get the code again
# auth_url, _ = flow.authorization_url(prompt='consent')
# print("Please visit this URL to authorize:")
# print(auth_url)
# code = input("Enter authorization code: ")
# flow.fetch_token(code=code)
# creds = flow.credentials

# Since the previous cell already has the authorization code, we can reuse the creds object
# Make sure to run the previous cell (ff42968b) first to define the creds object

# Assuming creds is defined in the previous cell, use it to build the service
service = build('gmail', 'v1', credentials=creds)


# Fetch recent 5 emails for processing
results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])
email_bodies = []

for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    parts = payload.get("parts")

    # Handle different payload structures
    email_text = "" # Initialize as empty string to concatenate parts
    if payload.get("body") and payload['body'].get('data'):
        data = payload['body'].get('data')
        if data:
            byte_code = base64.urlsafe_b64decode(data)
            email_text += byte_code.decode("utf-8")
    elif parts:
        for part in parts:
            if part.get("body") and part['body'].get('data'):
                data = part['body'].get('data')
                byte_code = base64.urlsafe_b64decode(data)
                email_text += byte_code.decode("utf-8") # Concatenate text parts

    if email_text: # Only append if email_text is not empty
        email_bodies.append(email_text)


print("✅ Successfully fetched", len(email_bodies), "emails for AI analysis.")


✅ Successfully fetched 5 emails for AI analysis.


In [ ]:
#Cell5
import google.generativeai as genai
from google.colab import userdata
import os

# Load Gemini API Key securely from Colab Secrets
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Configure Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Define the system prompt
system_prompt = """
You are an Enterprise Smart Notification Agent for campus automation.

Your task for each email:
1. Generate a 5-line summary.
2. Extract action deadlines and tasks.
3. Classify category: Academic, HR, Finance, IT, General.
4. Assign urgency: Critical, High, Normal.
5. Extract: Action required? Yes/No.

Output strictly in format:
Summary: <summary>
Category: <category>
Urgency: <urgency>
Action: <yes/no>
"""

# Load the Gemini model
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# Smart Notification Agent function
def smart_notification_agent(email_text):
    prompt = f"{system_prompt}\n\nEmail:\n{email_text}"
    response = model.generate_content(prompt)
    return response.text.strip()

# Loop over fetched emails (assume email_bodies is a list of email texts)
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Email {idx+1} =====")
    result = smart_notification_agent(email_text)
    print(result)


===== Email 1 =====
Summary: Omar Hussain Shaikh has successfully enrolled in a new course. The course is "Integrate Generative AI Into Your Data Workflow" on Google Cloud Skills Boost. You can sign in to your account to access the course content. This notification confirms your successful start. A direct sign-in link is provided.
Category: Academic
Urgency: Normal
Action: Yes

===== Email 2 =====
Summary: An SSH key named "wsl-ubuntu-retro-game" has been added to your account. The SHA256 fingerprint for this new key is provided for your reference. This notification informs you of recent account activity. If this key addition was unauthorized, it could pose a security risk. You can remove the key and disable access via the provided GitHub settings link.
Category: IT
Urgency: High
Action: Yes

===== Email 3 =====
Summary: A new fine-grained personal access token (wsl-ubuntu-retro-game) has been created on your GitHub account. This notification requires you to verify its legitimacy and 

In [ ]:
#cell6

import gspread
from google.oauth2.service_account import Credentials

# Your service account file (already uploaded to Colab)
SERVICE_ACCOUNT_FILE = '/content/serviceaccount.json'  # Replace if you used a different filename

# Define Sheets API scope
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# Authenticate using Service Account
sheet_creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES_SHEETS)

client = gspread.authorize(sheet_creds)

# Connect to your Google Sheet
spreadsheet_id = '10AV0X7nHqPsofsGYqnTNjNFZGP8z3JcpbGjGVOIWeMM'  # Replace with your actual spreadsheet ID
sheet = client.open_by_key(spreadsheet_id).sheet1

# (Optional) Write header row if not already written
sheet.append_row(["Date", "Sender", "Subject", "Summary", "Category", "Urgency", "ActionRequired"])

{'spreadsheetId': '10AV0X7nHqPsofsGYqnTNjNFZGP8z3JcpbGjGVOIWeMM',
 'updates': {'spreadsheetId': '10AV0X7nHqPsofsGYqnTNjNFZGP8z3JcpbGjGVOIWeMM',
  'updatedRange': 'Sheet1!A1:G1',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [ ]:
#cell7 Laststep
import datetime

# Example values for testing (replace with real email content later)
# Note: In a real scenario, you would ideally get sender and subject from the fetched email metadata.
# date_today = datetime.datetime.now().strftime("%d %b") # Use full date for better tracking
# sender = "HR Dept"
# subject = "Salary Update"

# Loop through all fetched emails
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Processing Email {idx+1} =====")

    # Assuming you have fetched email metadata (like sender, subject, date) in the email fetching step
    # Replace with actual metadata from your fetched emails
    # For now, using placeholder values
    date_today = datetime.datetime.now().strftime("%Y-%m-%d") # Using YYYY-MM-DD format
    sender = f"Sender {idx+1}" # Placeholder
    subject = f"Subject {idx+1}" # Placeholder


    result = smart_notification_agent(email_text)
    print(result)

    # Parse the LLM output more robustly
    lines = result.strip().split('\n')
    summary, category, urgency, action = "", "", "", ""

    for line in lines:
        if line.startswith("Summary:"):
            summary = line.split(":", 1)[1].strip()
        elif line.startswith("Category:"):
            category = line.split(":", 1)[1].strip()
        elif line.startswith("Urgency:"):
            urgency = line.split(":", 1)[1].strip()
        elif line.startswith("Action:"):
            action = line.split(":", 1)[1].strip()

    # Insert into Google Sheet
    try:
        sheet.append_row([date_today, sender, subject, summary, category, urgency, action])
        print("✅ Successfully inserted data into Google Sheet.")
    except Exception as e:
        print(f"❌ Error inserting data into Google Sheet: {e}")



===== Processing Email 1 =====
Summary: Omar hussain Shaikh has successfully started the course "Integrate Generative AI Into Your Data Workflow". This course is offered on the Google Cloud Skills Boost platform. You are encouraged to sign in to your account to access and review the course content. This email serves as a confirmation of your course enrollment.
Category: Academic
Urgency: Normal
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing Email 2 =====
Summary: An SSH key named "wsl-ubuntu-retro-game" with a specific SHA256 was added to your account. This is a security notification about the new key. Please review this addition for authorization. If the key was added in error or is unauthorized, it may indicate a security breach. You can remove the key and disable access via the provided GitHub settings link. Immediate action is advised if unauthorized.
Category: IT
Urgency: High
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Proces

In [ ]:
#cell7 Laststep
import datetime

# Example values for testing (replace with real email content later)
# Note: In a real scenario, you would ideally get sender and subject from the fetched email metadata.
# date_today = datetime.datetime.now().strftime("%d %b") # Use full date for better tracking
# sender = "HR Dept"
# subject = "Salary Update"

# Loop through all fetched emails
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Processing Email {idx+1} =====")

    # Assuming you have fetched email metadata (like sender, subject, date) in the email fetching step
    # Replace with actual metadata from your fetched emails
    # For now, using placeholder values
    date_today = datetime.datetime.now().strftime("%Y-%m-%d") # Using YYYY-MM-DD format
    sender = f"Sender {idx+1}" # Placeholder
    subject = f"Subject {idx+1}" # Placeholder


    result = smart_notification_agent(email_text)
    print(result)

    # Parse the LLM output more robustly
    lines = result.strip().split('\n')
    summary, category, urgency, action = "", "", "", ""

    for line in lines:
        if line.startswith("Summary:"):
            summary = line.split(":", 1)[1].strip()
        elif line.startswith("Category:"):
            category = line.split(":", 1)[1].strip()
        elif line.startswith("Urgency:"):
            urgency = line.split(":", 1)[1].strip()
        elif line.startswith("Action:"):
            action = line.split(":", 1)[1].strip()

    # Insert into Google Sheet
    try:
        sheet.append_row([date_today, sender, subject, summary, category, urgency, action])
        print("✅ Successfully inserted data into Google Sheet.")
    except Exception as e:
        print(f"❌ Error inserting data into Google Sheet: {e}")



===== Processing Email 1 =====
Summary: Omar hussain Shaikh has successfully started the course "Integrate Generative AI Into Your Data Workflow". This learning program is hosted on Google Cloud Skills Boost. The email confirms your enrollment in the specified course. You can sign in using the provided link to check your course details. This is an informational notification from Google Cloud Skills Boost Support.
Category: Academic
Urgency: Normal
Action: No
✅ Successfully inserted data into Google Sheet.

===== Processing Email 2 =====
Summary: A new SSH key named "wsl-ubuntu-retro-game" was recently added to your account. The key's SHA256 fingerprint is also provided for your reference. This notification is to inform you of the account change. If you did not authorize this key, it may indicate a security concern. You can remove the key and disable access via your GitHub settings.
Category: IT
Urgency: High
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing